In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

# ---- CONFIG ----
DATA_DIR = "REPLACE_WITH_YOUR_PATH"  # <-- SEN BURAYA KENDİ VERİSETİ YOLUNU YAZACAKSIN

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor()
])

train_ds = ImageFolder(f"{DATA_DIR}/train", transform=transform)
val_ds   = ImageFolder(f"{DATA_DIR}/val", transform=transform)
test_ds  = ImageFolder(f"{DATA_DIR}/test", transform=transform)

train_dl = DataLoader(train_ds, batch_size=16, shuffle=True)
val_dl   = DataLoader(val_ds, batch_size=16)
test_dl  = DataLoader(test_ds, batch_size=16)

len(train_ds), len(val_ds), len(test_ds)

In [ ]:
class CNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3,16,3,padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(16,32,3,padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32,64,3,padding=1), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.fc = nn.Linear(64*16*16, num_classes)

    def forward(self,x):
        x = self.net(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

model = CNN(num_classes=len(train_ds.classes)).to(device)
model

In [ ]:
opt = optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

def train_epoch():
    model.train()
    total, correct, total_loss = 0, 0, 0
    for img, lbl in train_dl:
        img, lbl = img.to(device), lbl.to(device)
        opt.zero_grad()
        out = model(img)
        loss = loss_fn(out, lbl)
        loss.backward()
        opt.step()
        total += lbl.size(0)
        correct += (out.argmax(1)==lbl).sum().item()
        total_loss += loss.item()
    return total_loss/len(train_dl), correct/total

def eval_epoch():
    model.eval()
    total, correct, total_loss = 0, 0, 0
    with torch.no_grad():
        for img, lbl in val_dl:
            img, lbl = img.to(device), lbl.to(device)
            out = model(img)
            loss = loss_fn(out, lbl)
            total += lbl.size(0)
            correct += (out.argmax(1)==lbl).sum().item()
            total_loss += loss.item()
    return total_loss/len(val_dl), correct/total

In [ ]:
epochs = 5
for e in range(epochs):
    tl, ta = train_epoch()
    vl, va = eval_epoch()
    print(f"Epoch {e+1}: Train Acc={ta:.3f}, Val Acc={va:.3f}")